## The gears of neural networks: tensor operations

所有计算机的程序，最终的可以简化为二进制运算。  
与此类似，神经网络所学到的变化也可以简化为数值数据上的一些张量运算(tensor operations)。  
在之前的2.1的例子里:  
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))  
可以理解为，输入一个2D张量，返回另一个2D张量。因此这个函数如下所示(其中w是一个2D张量,b是一个向量):  
output=relu(dot(w,input)+b)  
relu(x)=max(x,0) # 该步骤的目的是去除矩阵里的负数

### 2.3.1 Element-wise operations
逐元素运算

In [ ]:
import numpy as np

In [ ]:
def naive_relu(x):
    assert len(x.shape) == 2 # x是一个Numpy的2D张量
    x = x.copy() #避免覆盖输入张量
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

In [ ]:
def naive_add(x, y): #定义两个矩阵相加
    assert len(x.shape) == 2 # x和y 是一个Numpy的2D张量
    assert x.shape == y.shape
    x = x.copy()  #避免覆盖输入张量
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

In [ ]:
x= np.array([[-1,2,3,4],
            [2,-3,4,5],
            [3,4,-5,-6]])
y=x= np.array([[-1,-2,3,4],
            [2,-3,4,-5],
            [3,4,-5,-6]])
x*y

In [ ]:
import time

x = np.random.random((20, 100))
y = np.random.random((20, 100))

t0 = time.time()
for _ in range(1000):
    z = x + y
    z = np.maximum(z, 0.)
print("Took: {0:.2f} s".format(time.time() - t0)) # take0.01s

In [ ]:
t0 = time.time()
for _ in range(1000):
    z = naive_add(x, y)
    z = naive_relu(z)
print("Took: {0:.2f} s".format(time.time() - t0)) # take 2.08s

### 2.3.2 Broadcasting
广播
上一节是两个形状相同的2D张量相加，如果将两个形状不同的张量相加，会发生什么?  
如果没有奇异的话，较小的张量会被广播(broadcast)，以匹配较大张量的形状。广播包括以下两步:
1. 向较小的张量添加轴(叫做广播轴)，使其ndim与较大的张量相同。
2. 将较小的张量沿着新轴重复，使其形状与较大的张量相同。

In [ ]:
import numpy as np
X = np.random.random((32, 10))
y = np.random.random((10,))

In [ ]:

y = np.expand_dims(y, axis=0)

In [ ]:
y

In [ ]:
Y = np.concatenate([y] * 32, axis=0)

In [ ]:
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[j]
    return x

In [ ]:
import numpy as np
x = np.random.random((64, 3, 32, 10)) # x是(64,3,32,10)的随机张量
y = np.random.random((32, 10)) # y是形状(32,10)的随机张量
z = np.maximum(x, y) # 输出的z是(64，3，32，10)的随机张量

### 2.3.3 Tensor product
点积运算，也叫张量积(tensor product)，它与逐元素的乘积是不一样的。  
一般在Numpy,Keras, TensorFlow中，用*表示逐元素乘积，用 dot来表示点积。

In [ ]:
x = np.random.random((32,))
y = np.random.random((32,))
z = np.dot(x, y)

In [ ]:
print(z)
print(x*y)
print(sum(x*y))

点积就是逐元素乘积的总和。

In [ ]:
#两个向量的点积
def naive_vector_dot(x, y):
    assert len(x.shape) == 1 
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]
    z = 0.
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z
# 两个向量的点积是一个标量

In [ ]:
def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i] += x[i, j] * y[j]
    return z
#一个矩阵和一个向量的点积是一个向量，每个元素是y和x的每一行之间的点积。
#如果两个张量里有一个ndim>1,则dot运算不再对称，dot(x,y) != dot(y,x)

In [ ]:
(a,b,c,d).(d,) -> (a,b,c)
(a,b,c,d).(d,e) -> (a,b,c,e)

### 2.3.4 Tensor reshaping
张量变形

In [52]:
x = np.array([[0., 1.],
             [2., 3.],
             [4., 5.]])
x.shape

(3, 2)

In [49]:
x = x.reshape((6, 1))
x

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.]])

In [50]:
x = x.reshape((2, 3))
x

array([[0., 1., 2.],
       [3., 4., 5.]])

变形时顺序为先满足每一行，再满足列

In [51]:
x = np.zeros((300, 20)) # 创建一个形状为(300,20)的零矩阵
x = np.transpose(x)
x.shape

(20, 300)

In [53]:
np.transpose(x)

array([[0., 2., 4.],
       [1., 3., 5.]])

### 2.3.5 Geometric interpretation of tensor operations
张量运算的几何解释

一个二维向量旋转theta角，可以通过一个R= [u,v]的二维矩阵做点积实现，其中  
u= [cos(theta),sin(theta)],  
v=[-sin(theta),cos(theta)]

### 2.3.6 A geometric interpretation of deep learning
深度学习的几何解释

想象有两张彩纸：一张红色，一张蓝色。将其中一张纸放在另一张上。现在将两张纸一起揉成小球。这个皱巴巴的纸球就是你的输入数据，每张纸对应于分类问题中的一个类别。神经网络（或者任何机器学习模型）要做的就是找到可以让纸球恢复平整的变换，从而能够再次让两个类别明确可分。通过深度学习，这一过程可以用三维空间中一系列简单的变换来实现，让纸球恢复平整就是机器学习的内容：为复杂的、高度折叠的数据流形找到简洁的表示。现在你应该能够很好地理解，为什么深度学习特别擅长这一点：它将复杂的几何变换逐步分解为一长串基本的几何变换，这与人类展开纸球所采取的策略大致相同。深度网络的每一层都通过变换使数据解开一点点——许多层堆叠在一起，可以实现非常复杂的解开过程。